# Примитивно рекурсивные функции

Понятие **примитивно рекурсивной функции** является индуктивным. Оно состоит из указания класса **базовых функций** и двух операторов (композиции и примитивной рекурсии), позволяющих строить новые примитивно рекурсивные функции на основе уже имеющихся.

Задача вычисления экспоненты может быть сведена к задаче вычисления произведения, а это в свою очередь, к сложению, а сложение может быть сведено к последовательному прибавлению единиц. Последнее может быть реализовано функцией следования (см. далее).

Примитивно рекурсивные функции важны тем, что заведомо являются вычислимыми – они могут быть вычислены с использованием конечного числа шагов и всегда заканчивают выполнение. Однако, не все вычислимые функции сводятся к примитивно рекурсивным функциям. В конце описывается класс рекурсивных функций, который, согласно тезису Чёрча, покрывает множество всех вычислимых функций.

## Базовые функции

Рассмотрим стандартный набор базовых функций, которые принимают в качестве аргументов и возвращают значения из множества целых неотрицательных чисел $\mathbb N_0$. В общем случае могут быть определены другие базовые функции на множестве произвольной природы.

**Нулевая функция** (*zero function*) $z(n_1, \ldots, n_k)$ всегда возвращает $0$: $\; z(5) = 0$.

**Функция следования** (*successor function*) $s(n)$ одноместная функция, которая возвращает элемент, следующий за аргументом. Для целого аргумента $n$ возвращается число $n+1$: $\; s(3)=4$.

**Проективная функция** или **тождественная функция** (*projection function* или *identity function*) $I_i^k$ – $k$-местная функция, которая возвращает аргумент с индексом $i$, где $0<i<k$:
$$
I_i^k(n_1, \ldots, n_k) = n_i
$$
При $k=1$ можно записать $I(n)=n$.

In [2]:
def z(*args, **kwargs):
    return 0

def I(v: int|tuple|list, index=None):
    """Identity function. 
    If v is array index must be given"""
    if not index is None:
        return v[index]
    elif isinstance(v, int):
        return v
    elif len(v) == 1:
        return v[0]
    else:
        raise TypeError
    
def get_id_function(i=None, k=None):
    """Identity function generator"""
    if i is None or k==1:
        return I
    elif i >= k or k < 0 or i < 0:
        raise ValueError ("i must be less than k and both non-negative")
    else:
        def f(*v):
            if len(v) != k:
                raise ValueError (f"len(v) must be equal {k}")
            return I(v, i)
        return f

def s(x: int):
    """Successor function"""
    if isinstance(x, int):
        return x+1
    else:
        raise TypeError ("x must be int")

## Операции

### Композиция

**Композиция** (*composition*) или **подстановка** (*substitution*). Пусть $f$ – функция от $m$ переменных, а $g_i$ – функции от $n$ переменных. Тогда функция $h$, определяемая равенством
$$
h(x_1, \ldots, x_n) = f(g_1(x_1, \ldots, x_n), \ldots, g_m(x_1, \ldots, x_n))
$$
называется композицией функций $f, g_1, \ldots, g_m$. Коротко можно записать через оператор композиции
$$
h = f \circ (g_1, \ldots, g_m) = \mathrm{Cn}[f, g]
$$

In [3]:
def Cn(f, *gs):
    """Composition"""
    def h(*args):
        interim = []
        for g in gs:
            interim.append(g(*args))
        return f(*interim)
    return h

В общем случае, функция $g_i$ не обязана быть функцией от всех аргументов $x_1, \ldots, x_n$. Но для учета этого можно отсавить прежнуюю запись, предполагая, что внутри функции будут отобраны нужные аргументы при помощи проективных функций. 

**Пример**

Предположим, мы хотим реализовать функцию $h(x, y) = x + 2$. Можно представить ее в виде композиции функций $I_1^2$ и $s$:
$$
h = s \circ s \circ I_1^2 = \mathrm{Cn}[s, \mathrm{Cn}[s, I_1^2]]
$$

In [4]:
id_1_2 = get_id_function(0, 2)
g = Cn(s, id_1_2)
h = Cn(s, g)

In [5]:
h(14, 5)

16

### Примитивная рекурсия

**Примитивная рекурсия** (*primitive recursion*). Пусть $f$ – функция от $m$ переменных, а $g$ – функции от $m+2$ переменных. Функции $f$ и $g$ определены как примитивно рекурсивные функции. Тогда функция $h$ определяемая операцией примитивной рекурсии из $f$ и $g$, также примитивно рекурсивна. Примитивная рекурсия задается уравнением:
$$
h(x_1, \ldots, x_m, 0) = f(x_1, \ldots, x_m);\\
h(x_1, \ldots, x_m, s(y)) = g(x_1, \ldots, x_m, y, h(x_1, \ldots, x_m, y))
$$
$y$ – номер шага итераций. Функцию $f$ можно рассматривать как исходную функцию в начале итерационного процесса. Функция $g$ принимает $m$ переменных $x_i$, номер шага итераций $y$ и значение функции $h$ на текущем шаге итерации, и возвращает значение функции $h$ на следующем шаге итерации. При $m=1$:
$$
h(x, 0) = f(x);\\
h(x, s(y)) = g(x, y, h(x, y))
$$
Функцию $h$, определяемую путем примитивной рекурсии из функций $f$ и $g$, обозначим
$$
h= \mathrm{Pr}[f, g]
$$

In [6]:
def Pr(f, g):
    def h(x, y):
        # print(f"{y = }")
        if y == 0:
            # print('terminate')
            return f(x)
        return g(x, y-1, h(x, y-1))
    return h

В данном случае оператор `-` используется в коде для имитации вот этого участка: $h(x, s(y)) = g(x, y, h(x, y))$, так как в Python нет возможности задать непосредственно в этом виде.

Функции, которые могут быть получены из базовых функций при помощи композиции и рекурсии называются **примитивно рекурсивными функциями** (*primitive recursive functions*). Все функции, получаемые путем композиции и рекурсии из всюду определенных (*total*) функций также являются всюду определенными. Так как стандартные базовые функции являются всюду определенными на множестве неотрицательных целых чисел, то и все примитивно рекурсивные функции являются всюду определенными в этой области. Таким образом примитивно рекурсивные функции всегда завершаются и дают результат за конечное время, на всех возможных входых значениях.

Примитивно рекурсивные функции не охватывают частично определенные (*partial*) вычислимые функции. Более того, не все всюду определенные вычислиымые функции относятся к классу примитивно рекурсивных функций (примером может служить функция Аккермана, которая является всюду определенной). Однако, обобщение, путем добавления еще одной операции (минимизации), приводит к классу рекурсивныех функций, который позволяет охватить как всюду определенные, так и частично определенные вычислимые функции.

## Примеры

### Сложение

Функция **сложения** $\mathrm{add}$ двух натуральных чисел. Определим $f(x)$ как $I(x)$, а $g(x)$ как композицию функций $s$ и $I_3^3$, т.е. $\mathrm{Cn}[s, I_3^3]$, чтобы отобрать третий аргумент, переданный в $g$ и прибавить к нему 1:
$$
\mathrm{add}(x, 0) = I(x) \\
\mathrm{add}(x, s(y)) = s \circ I_3^3(x, y, \mathrm{add}(x, y))
$$
Таким образом можно записать:
$$
\mathrm{add} = \mathrm{Pr}[I, s \circ I_3^3] = \mathrm{Pr}[I, \mathrm{Cn}[s, I_3^3]]
$$

In [7]:
f = get_id_function()
g = Cn(s, get_id_function(2, 3))
add = Pr(f, g)

In [8]:
add(14, 4)

18

### Умножение

Функция **умножения** $\mathrm{mul}$ двух натуральных чисел. Определим $f(x)$ как $z(x)$, а $g$ определим как композицию функции $\mathrm{add}$ и функций $I_1^3, I_3^3$, чтобы производилось сложение первого и третьего аргументов функции $g$:
$$
\mathrm{mul}(x, 0) = z(x) \\
\mathrm{mul}(x, s(y)) = \mathrm{add} \circ (I_1^3, I_3^3)
$$
Таким образом можно записать:
$$
\mathrm{mul} = \mathrm{Pr}[z, \mathrm{add} \circ (I_1^3, I_3^3)] = \mathrm{Pr}[z, \mathrm{Cn}[\mathrm{add}, I_1^3, I_3^3]]
$$

In [9]:
f = z
g = Cn(add, get_id_function(0, 3), get_id_function(2, 3))
mul = Pr(f, g)

In [17]:
mul(3, 4)

12

### Exp

In [11]:
def f(x):
    return s(z(x))
g = Cn(mul, get_id_function(0, 3), get_id_function(2, 3))
exp = Pr(f, g)

exp(2, 10)

1024

### Гипероператоры

Продолжая подобным образом эту серию мы получаем мы получаем операцию **тетрации** (суперэкспоненту), которая представляет собой стек из экспонент $x^{x^{x^{.^{.^{.}}}}}$ (всего в стеке $y$ "иксов"). Эту операцию удобнее записать используя оператор экспоненты  $\uparrow$ и оператор тетрации ($\uparrow \uparrow$):

$$
x \uparrow \uparrow y= x \uparrow x \uparrow x \dots \uparrow x
$$

Данную серию можно неограниченно продолжать, и так как мы каждый раз получаем функции, на основе примитивно рекурсивных функций, используя операции композиции и примитивной рекурсии, все функции серии являются примитивно рекурсивными.

Обобщенно, операторы этой серии называются гипероператорами $n$-го порядка. Сложение – гипероператор первого порядка, умножение – второго, экспонента – третьего. Далее идет тетрация $\uparrow \uparrow$, пентация $\uparrow \uparrow \uparrow$, гексация $\uparrow^4$ и тд.

Другими примерами примитивно рекурсивных функий могут служить: факториал, предшествующее число (кроме случая $0$), ограниченное вычитание. Обратные функции вообще говоря не сводятся к примитивно рекурсивным функциям. Для их реализации требуется операция минимизации.

### Рекурсивные функции от одной переменной

Если рассматривается единственная переменна $y$, то схема принимает вид:
$$
h(0) = f() \\
h(s(y)) = g(y, h(y))
$$
В этом случае $x_i$ не появляется вовсе, а функция $f$ нуля переменных есть некоторая константа $s(0)$ или $s(s(0))$ и тд.

In [18]:
def Pr(f, g):
    def h(y):
        if y == 0:
            return z(y)
        return g(y-1, h(y-1))
    return h

 Другой способ – использовать предыдущую функцию `Pr` с фиктивным (*dummy*) $x$.

### Prev

Получим схему для функции от одной переменной, возвращающей предыдущее значение. Если значение аргумента 0, то возвращается 0:
$$
\mathrm{pd}(0) = 0 \\
\mathrm{pd}(s(y)) = I_2^3
$$

# Рекурсивные функции

Для того, чтобы можно было охватить также и множество частично определенных фунций, необходимо расширить класс примитивно рекурсивных функций, добавив еще одну операцию для продуцирования новых функций.

Интуитивно, частично определенная функция (*partial function*) $f$ является вычислимой, если может быть задан список определенных и явных инструкций, следуя которым, в случае применения к любому $x$, принадлежащему области определения функции, можно получить значение $f(x)$, однако, в случае применения к $x$, не принадлежащему области определения функции, следование инструкциям будет продолжаться вечно, не приводя к результату, либо завершится не дав результат. Это касается и функций от многих переменных.

Класс общерекурсивных (или просто рекурсивных) функций содержит в себе класс примитивно рекурсивных функций, к которым сводятся всюду определенные функции, и класс частично рекурсивиных функций, к которым сводятся частично определенные функции.

Класс рекурсивных функций получается из класса примитивно рекурсивных функций путем добавления операции минимизации для продуцирования новых функций.

## Минимизация

Пусть дана функция $f$ от $n+1$ аргументов $f(x_1, \ldots, x_n, y)$. Операция **минимизации** (*minimization*) дает всюду или частично определенную функцию $h$ от $n$ аргументов $h(x_1, \ldots, x_n)$:
$$
\mathrm{Mn}[f](x_1, \ldots, x_n) = 
\begin{cases}
y & \text{если } f(x_1, \ldots, x_n, y)=0, \\ 
& \text{и для всех } t<y \\
& f(x_1, \ldots, x_n, t) \text{ определена и не равна нулю} \\
\text{undefined} & \text{если если не существует такого } y
\end{cases}
$$
Функция $h$ возвращает минимальное значение последнего аргумента функции $f$, при котором $f$ принимает значение $0$, при условии, что функция определена при всех значениях последнего аргумента, которые меньше этого минимального значения.

Для $h(\mathbf x)=h(x_1, \ldots, x_n)=\mathrm{Mn}[f]$ мы вычисляем последовательно $f(\mathbf x, 0), f(\mathbf x, 1), f(\mathbf x, 2), \ldots$, останавливаясь при достижении такого $y$, при котором $f(\mathbf x, y) = 0$. Для $\mathbf x$, функция $h$ может оказаться неопределенной по двум причинам. Во-первых, может оказаться, что все значения последовательности $f(\mathbf x, 0), f(\mathbf x, 1), f(\mathbf x, 2), \ldots$ определены, нет нулевых значений. Во-вторых, может оказаться, что значения последовательности $f(\mathbf x, 0), f(\mathbf x, 1), \ldots, f(\mathbf x, i-1)$ определены, но среди них нет нулевых, а $f(\mathbf x, i)$ уже не определена. В этих случаях попытка вычислить $h(\mathbf x)$ будет вовлекать в процесс, который будет продолжнаться вечно, не давая результата. 

В случае, когда $f$ является всюду определенной, мы не беспокоимся о второй причине неопределенности $h$ и определение выще может быть записано так:
$$
\mathrm{Mn}[f](x_1, \ldots, x_n) = 
\begin{cases}
\text{наименьший }y, \text{при котором} \\ \quad f(x_1, \ldots, x_n, y)=0
& \text{если существует такой } y\\ 
\text{undefined} & \text{в противном случае}
\end{cases}
$$

Всюду определенная функция $f$ называется **регулярной** (*regular*), если для каждого $x_1, \ldots, x_n$ существует такой $y$, при котором $f(x_1, \ldots, x_n, y)=0$. В случае, когда $f$ является регулярной функцией, $\mathrm{Mn}[f]$ будет всюду определенной функцией. Более того, если $f$ является всюду определенной функцией, то $\mathrm{Mn}[f]$ будет всюду определенной (при любых $\mathbf x$) функцией в том и только в том случае, когда $f$ является регулярной. В качестве примера можно рассмотреть функции сложения и умножения: умножение является регулярной функцией, а сложение - нет, так как $f(x,y)=x+y=0$ имеет место только при $x=y=0$.

Функции, которые могут быть получены из базисных функций $z, s, I_i^k$ при помощи операций $\mathrm{Cn, Pr, Mn}$ называются рекурсивными функциями (*recursive functions*). В литературе обычно под **рекурсивной функцией** подразумевают всюду определенную рекурсивную функцию, а под **частичной рекурсивной функцией** (*partial recursive function*) подразумевают в общем как частично определенную, так и всюду определенную рекурсивной функции (всюду определенная функция является частным случаем частично определенной функции).

Операция минимизации позволяет получить обратные функции.

## Пример

Определим фунцию $\mathrm{Div}(x, y)$ – целая часть от деления $x$ на $y$. Исходная функция может быть выбрана так:

$$
f(x, y, z) = \left[ y(z+1) < x \right]
$$

Функция всюду определена на $\mathbb N_0$, область значений $\{0, 1\}$. Значение функции будет равно $1$ для всех значений $z$, при которых значение выражения $y(z+1)$ не превышает $x$. Если увеличивать $z$, то как только значение выражения $y(z+1)$ превысит $x$ функция $f$ вернет $0$. Это наименьшее $z$, при котором $f$ обращается в $0$ и будет целой частью от деления $x$ на $y$:

$$
z = \left\lfloor \frac{x}{y} \right\rfloor\
$$

Таким образом,

$$
\mathrm{Div}(x, y) = \mathrm{Mn}[f] = \mathrm{Mn}\left[ y(z+1) < x \right]
$$

# Тезис Чёрча
Согласно тезису Чёрча все вычислимые всюду определенные функции являются рекурсивными функциями (гипотиза о том, что все вычислимые частично определенные функции являются рекурсивными, известна как расширенная версия тезиса Чёрча). На сегодняшний день, тезис Чёрча представляет собой просто гипотезу. Правдоподобность тезиса подтверждается значительным числом вычислимых функций, для которых показано, что они являются рекурсивными. Однако это конечно не доказывает правильность тезиса.

In [6]:
from pytube import YouTube
from pytube.exceptions import RegexMatchError, VideoUnavailable
import time

# Function to download YouTube video
def download_youtube_video(url, output_path='.'):
    try:
        # Create a YouTube object
        yt = YouTube(url)
        
        # Get the highest resolution stream available
        stream = yt.streams.get_highest_resolution()
        
        # Download the video
        print(f"Downloading {yt.title}...")
        stream.download(output_path=output_path)
        print(f"Downloaded successfully: {yt.title}")
    except RegexMatchError:
        print("The video URL provided is incorrect.")
    except VideoUnavailable:
        print("The video is unavailable. Please check the URL or try another video.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

video_url = 'https://www.youtube.com/watch?v=vVKuvGoMxjA&t'
download_directory = './downloads'

# Attempt to download the video, retrying if a bad request occurs
retries = 3
for attempt in range(retries):
    try:
        download_youtube_video(video_url, download_directory)
        break
    except Exception as e:
        print(f"Attempt {attempt + 1} failed: {e}")
        time.sleep(5)  # Wait for 5 seconds before retrying
    if attempt == retries - 1:
        print("Failed to download the video after multiple attempts.")

An unexpected error occurred: HTTP Error 400: Bad Request


In [4]:
!pip install --upgrade pytube